# Deploy Yolo Model on AWS

---

- Reference:
  - [HowTo: deploying YOLOv8 on AWS Lambda](https://www.trainyolo.com/blog/deploy-yolov8-on-aws-lambda)


## Optional - Google CodeLab


In [12]:
# !pip install ultralytics
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Converting YOLOv8 to ONNXz


In [7]:
from ultralytics import YOLO

# load model
MODEL_PATH = './model/car_plate/best.pt'
model = YOLO(MODEL_PATH)

# # export model
model.export(format="onnx", dynamic=True)

Ultralytics YOLOv8.1.9 🚀 Python-3.11.5 torch-2.2.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'model\car_plate\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 0.7s, saved as 'model\car_plate\best.onnx' (11.5 MB)

Export complete (2.5s)
Results saved to C:\Users\simon\iCloudDrive\Documents\Github\VehiclePlateDetector\model\car_plate
Predict:         yolo predict task=detect model=model\car_plate\best.onnx imgsz=640  
Validate:        yolo val task=detect model=model\car_plate\best.onnx imgsz=640 data=datasets/config.yaml  
Visualize:       https://netron.app


'model\\car_plate\\best.onnx'

- Local Jupyter Notebook
  - ONNX file: `/deploy/model.onnx`


## Install AWS SAM CLI

- rel:
  - https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/install-sam-cli.html#install-sam-cli-instructions


## Define Lambda function


In [11]:
from yolo_onnx.yolov8_onnx import YOLOv8
import json
import base64
from io import BytesIO
from PIL import Image

# Definec a global parameter for the Initialized YOLOv8 object detector
# global parameter: remains in memory during the lifetime of the function; not be re-initialized for each function all
yolo_detector = YOLOv8('model.onnx')


def main(event, context):

    # Get request body
    body = json.loads(event['body'])

    # get arguments
    image = body['image']               # encoded image
    size = body.get('size', 640)        # size
    conf_thres = body.get('conf_thres', 0.3)        # confidence threshold
    iou_thres = body.get('iou_thres', 0.5)          # IOU threshold

    # decoded
    img = Image.open(
        BytesIO(
            base64.b64decode(image.encode('ascii'))
        ))

    # predict
    pred_yolo = yolo_detector(
        img, size=size, conf_thres=conf_thres, iou_thres=iou_thres)

    # return prediction
    return {
        "statusCode": 200,
        "body": json.dumps({
            "detections": pred_yolo
        }),
    }

# return sample:
# [
#   {"bbox": [0, 264, 473, 1351], "score": 0.932, "class_id": 0},
#   {"bbox": [1286, 199, 1918, 1353], "score": 0.93, "class_id": 0},
#   {"bbox": [1047, 288, 1534, 1348], "score": 0.895, "class_id": 0},
#   {"bbox": [494, 433, 943, 1357], "score": 0.895, "class_id": 0},
#   {"bbox": [1021, 748, 1111, 901], "score": 0.679, "class_id": 41}
# ]

## Deploying the Lambda function
